# Load and Preprocess the Dataset

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from datetime import datetime
import matplotlib.pyplot as plt


# Paths to your dataset
train_dir = "asl_alphabet_train/"
test_dir = "asl_alphabet_test/"

# Define image size and batch size
img_size = (64, 64) 
batch_size = 32

# Data augmentation and preprocessing for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=0.2 
)

# Data preprocessing for test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,  # Ensure target_size is a tuple
    batch_size=batch_size,
    color_mode='rgb', 
    class_mode='categorical',
    subset='training'
)

# Load and preprocess validation data
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,  # Ensure target_size is a tuple
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    subset='validation'
)

# Load and preprocess test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,  # Ensure target_size is a tuple
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical'
)


Found 69600 images belonging to 1 classes.
Found 17400 images belonging to 1 classes.
Found 28 images belonging to 1 classes.


# Define the CNN Model

In [2]:
def create_model():
    model = Sequential([
        Input(shape=(64, 64, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Update this line
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Update this line
    return model

model = create_model()


# Train the Model

In [3]:
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator
)

Epoch 1/30


C:\Users\Suyash\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2175/2175 ━━━━━━━━━━━━━━━━━━━━ 300s 136ms/step - accuracy: 0.9975 - loss: 0.0030 - val_accuracy: 1.0000 - val_loss: 1.5399e-20
Epoch 2/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 293s 134ms/step - accuracy: 1.0000 - loss: 5.8713e-19 - val_accuracy: 1.0000 - val_loss: 1.1101e-20
Epoch 3/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 292s 134ms/step - accuracy: 1.0000 - loss: 6.8895e-22 - val_accuracy: 1.0000 - val_loss: 1.0221e-20
Epoch 4/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 292s 134ms/step - accuracy: 1.0000 - loss: 2.5793e-21 - val_accuracy: 1.0000 - val_loss: 4.5961e-21
Epoch 5/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 291s 134ms/step - accuracy: 1.0000 - loss: 2.0357e-20 - val_accuracy: 1.0000 - val_loss: 1.9810e-20
Epoch 6/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 292s 134ms/step - accuracy: 1.0000 - loss: 3.4373e-20 - val_accuracy: 1.0000 - val_loss: 8.8330e-21
Epoch 7/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 292s 134ms/step - accuracy: 1.0000 - loss: 2.5425e-19 - val_accuracy: 1.0000 - val_loss: 9.0649e-21
Epoch 8/30
2175/2175 

In [4]:
# Evaluate the model
validation_loss, validation_accuracy = model.evaluate(val_generator)
print(f'Validation Accuracy: {validation_accuracy}')


544/544 ━━━━━━━━━━━━━━━━━━━━ 30s 54ms/step - accuracy: 1.0000 - loss: 1.6921e-19
Validation Accuracy: 1.0


# Time Restiction

In [5]:
def is_within_time_window():
    current_time = datetime.now().time()
    start_time = datetime.strptime("18:00", "%H:%M").time()
    end_time = datetime.strptime("22:00", "%H:%M").time()
    return start_time <= current_time <= end_time

def predict_sign_language(image):
    if is_within_time_window():
        prediction = model.predict(image)
        return np.argmax(prediction, axis=1)
    else:
        return "Prediction not allowed outside 6 PM to 10 PM"


In [5]:
import tensorflow as tf
from tensorflow.keras.models import model_from_json

# Load the model architecture from the JSON file
with open("emotion_detector_model.json", "r") as json_file:
    model_json = json_file.read()

model = model_from_json(model_json)

# Load the weights into the model
model.load_weights(".weights.h5")


In [6]:
# Save the model
model_json = model.to_json()
with open("Sign_Language_model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(".weights.h5")
print("Model saved to disk")


Model saved to disk
